In [7]:
import sys
import os
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_meinhold')
sys.path.append('../../cofe-python-analysis-tools/utils_meinhold')
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_zonca')
sys.path.append('../../cofe-python-analysis-tools/utils_zonca')
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_zonca\\pointing')
sys.path.append('../../cofe-python-analysis-tools/utils_zonca/pointing')
#sys.path.append('D:\\software_git_repos\\greenpol')
sys.path.append('../')
#sys.path.append('D:\\software_git_repos\\greenpol\\telescope_control\\')
sys.path.append('../telescope_control')
#sys.path.append('D:\\software_git_repos\\greenpol\\telescope_control\\VtoT\\')
sys.path.append('../telescope_control/VtoT')
import realtime_gp as rt
import numpy as np
import datetime 
import h5py
import pandas as pd
from pointingtools import compute_parallactic_angle, altaz2ha 
from planets import getlocation
import warnings
from astropy.coordinates import AltAz, Angle, EarthLocation, ICRS, SkyCoord, frame_transform_graph
from astropy import units as u
import ephem
import matplotlib.pyplot as plt
import time
from datetime import datetime

import Tkinter,tkFileDialog

from Tkinter import *
import ttk

import pickle
import glob

import cPickle

%pylab

Using matplotlib backend: Qt4Agg
Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


In [8]:
def get_pointing_files(filelist=None):

    if filelist==None:
        root=Tkinter.Tk()
        filelist = list(tkFileDialog.askopenfilenames(\
        initialdir='D://software_git_repos/greenpol/telescope_control/data_aquisition/pointing_data/',parent=root,title='Choose a set of files'))
        root.destroy()
    filelist.sort()
    
    return filelist

In [9]:
def read_some_data(filelist=None):

    if filelist==None:
        root=Tkinter.Tk()
        filelist = list(tkFileDialog.askopenfilenames(\
        initialdir='D://software_git_repos/greenpol/telescope_control/data_aquisition/demod_data/',parent=root,title='Choose a set of files'))
        root.destroy()
    filelist.sort()
    
    dlist=[]
    for f in filelist:
        hf=h5py.File(f)
        dlist.append(hf['demod_data'])
    d=np.concatenate(dlist)
    hf.close() 

    datadict=d
        
    return datadict, filelist

In [10]:
def get_file_times(fld):
    startfile = fld[0][:65]+fld[0][71:-2]+'dat'
    endfile = fld[-1][:65]+fld[-1][71:-2]+'dat'
    
    #starttime = os.path.getctime(startfile)
    starttime= os.stat(startfile).st_mtime
    starttime = datetime.datetime.fromtimestamp(starttime)

    #endtime = os.path.getctime(endfile)
    endtime= os.stat(endfile).st_mtime
    endtime = datetime.datetime.fromtimestamp(endtime)
    
    return starttime, endtime

In [11]:
def fileStruct(n_array, chan, starttime, endtime):

    fpath = "D:/software_git_repos/greenpol/telescope_control/data_aquisition/level1"
    os.chdir(fpath)
    yrmoday = starttime.strftime('%Y%m%d')
    path = '-'.join((starttime.strftime('%H_%M_%S'), endtime.strftime('%H_%M_%S')))
    path = '-'.join((chan, path))
    print 'start time: ', starttime
    print 'end time: ', endtime
    print 'elapsed time: ', (endtime - starttime).total_seconds(), 'sec'
    if not os.path.exists(yrmoday):#this is the first file being created for that time
        os.makedirs(yrmoday)
        #set index to 0

    path = '/'.join((yrmoday,path))
    path = '.'.join((path,"h5"))
    with h5py.File(str(path).replace("pkl","h5"), mode="w") as f:
        f.create_dataset("data", data=n_array.to_records(index=False))

In [12]:
def round_fraction(number, res):
    amount = int(number/res)*res
    remainder = number - amount
    return amount if remainder < res/2. else amount+res

In [13]:
def convert_gpstime(starttime, gpstime, ltoffset=0, bits_to_ms=2**24, format='seconds', ttype = 'utc', singletime=False):

    #function to convert gpstime to UTC time or local time

    #universal time day
    ltoffset = ltoffset*60*60
    utcday = starttime + datetime.timedelta(0, ltoffset)

    #days since last sunday
    idx = (utcday.weekday() + 1) % 7

    #date of previous sunday in universal coord
    sunday = utcday - datetime.timedelta(idx) 

    syear = int(str(sunday)[:4])
    smonth = int(str(sunday)[5:7])
    sday = int(str(sunday)[8:10])
    
    sunday = datetime.datetime(syear, smonth, sday, 0, 0, 0)

    #seconds since last sunday
    sundaysec = (utcday - sunday).total_seconds()

    #convert to seconds
    bits_to_sec = bits_to_ms/1000
    
    #number of wraps so far
    numwraps = int(sundaysec/bits_to_sec)
    
    #time of last wrap
    gpsstarttime = sunday + datetime.timedelta(0, numwraps*bits_to_sec)
    
    #convert gps starttime to timestamp
    #gpsstarttime = (gpsstarttime-datetime.datetime(1970,1,1)).total_seconds()
    gpsstarttime = time.mktime(gpsstarttime.timetuple())
    
    if singletime == False:
        #find all wrap points in current data set
        gpsdiff = np.diff(gpstime)
        iwrap = np.where(gpsdiff < -2**24/1000/2)[0]
        #iwrap = np.where(abs(gpsdiff) > 2**24/1000/2)[0]

        #unwrap gpstime
        for w in iwrap:
            gpstime[w+1:] = gpstime[w+1:] + gpstime[w]

    if ttype == 'utc':
        ltoffset = 0
    #gpstime gives seconds since starting point
    dtime = gpsstarttime + gpstime - ltoffset

        
    if format == 'datetime':
        if singletime == False:
            t = []
            for i in range(len(dtime)):
                t.append(datetime.datetime.fromtimestamp(dtime[i]))
        else:
            t = datetime.datetime.fromtimestamp(dtime)
        
        return t, dtime
    
    else:
        return dtime

    '''
    dtime = []
     
    for i in range(len(gpstime)):
    #for i in range(0,1):
        
        if i>0:
            if abs(gpstime[i] - gpstime[i-1]) > 2**24/1000/2:
                print 'it got here'
                gpstime[i:] = gpstime[i:] + gpstime[i-1]
                
        #gpstime gives seconds since starting point
        t = gpsstarttime + datetime.timedelta(0,gpstime[i]) - datetime.timedelta(0, ltoffset)

        if format == 'seconds':
            timestamp = (t-datetime.datetime(1970,1,1)).total_seconds()
            dtime.append(timestamp)
        else:
            dtime.append(t)
          
    return dtime
    '''

In [14]:
ddict={}

In [15]:
dd, fld = read_some_data()

In [16]:
flp = get_pointing_files()
pp = rt.get_h5_pointing(flp)

In [17]:
combined = rt.combine_cofe_h5_pointing(dd,pp)

In [18]:
plot(combined['gpstime'])

In [19]:
starttime, endtime = get_file_times(fld)

print starttime
print endtime

2018-08-02 11:46:19.022157
2018-08-02 22:58:07.434666


In [22]:
chan = 'ch0'
var = 'Q'
LOCATION = 'Greenland'

#gain in kelvin per volt
changain = {'ch0':-3.58, 'ch4':-9.47, 'ch8':-11.1}

In [23]:
#offset between local time and utctime in hours
ltoffset = 0

#seconds since last sunday utc
gpstime = dd['rev']/1000#(combined['gpstime'])/1000

In [24]:
gpstime = dd['rev']/1000#(combined['gpstime'])/1000
dtime = convert_gpstime(starttime, gpstime, ltoffset, format = 'datetime', ttype = 'ltc')[0]
tdiffe = (dtime[-1] - endtime).total_seconds()
tdiffs = (dtime[0] - starttime).total_seconds()
print 'file start time: ', starttime
print 'gps converted start time: ', dtime[0]
print 'amount off in seconds: ', tdiffs
print 'file end time: ', endtime
print 'gps converted end time: ', dtime[-1]
print 'amount off in seconds: ', tdiffe

file start time:  2018-08-02 11:46:19.022157
gps converted start time:  2018-08-02 11:46:23.799000
amount off in seconds:  4.776843
file end time:  2018-08-02 22:58:07.434666
gps converted end time:  2018-08-03 03:38:16.406000
amount off in seconds:  16808.971334


In [76]:
print datetime.utcnow()

AttributeError: 'module' object has no attribute 'utcnow'

In [77]:
gpstime = dd['rev']/1000#(combined['gpstime'])/1000
dtime, utime = convert_gpstime(starttime, gpstime, ltoffset, format = 'datetime')

In [78]:
AZ = combined['az']
EL = combined['el'] 

location = getlocation(LOCATION)

t1 = time.time()
#create ra dec sky object
azel = SkyCoord(az = AZ, alt = EL, obstime = dtime, location = location, frame = 'altaz', unit='deg')
t2 = time.time()
print t2-t1
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    #convert from ra dec to az/el for pointing
    radec = azel.icrs
print time.time()-t2
ra = radec.ra.rad
dec = radec.dec.rad

1.92500019073
92.9619998932


In [79]:
AZ = np.radians(AZ)
EL = np.radians(EL)

In [80]:
lat = location.latitude.rad
ha = altaz2ha(EL, AZ, lat)
psi = compute_parallactic_angle(ha, lat, dec)

In [81]:
#need to figure out units on TIME in andreas test file

In [82]:
#h5data=pd.DataFrame({"THETA" : np.pi/2 - THETA })
h5data=pd.DataFrame({"TIME" : utime}) 
h5data["PHI"] = ra
h5data["THETA"] = np.pi/2 - dec
h5data["PSI"] = psi
h5data["FLAG"] = np.zeros(len(dtime)) 
h5data["TEMP"] = combined['sci_data'][chan]['T']*changain[chan]
h5data["Q"] = combined['sci_data'][chan]['Q']*changain[chan]
h5data["U"] = combined['sci_data'][chan]['U']*changain[chan]

In [83]:
fileStruct(h5data, chan, starttime, endtime)

start time:  2018-08-02 13:09:02.854072
end time:  2018-08-02 16:10:13.371830
elapsed time:  10870.517758 sec


In [51]:
figure()
plot(AZ)

In [5]:
chan = 'H1HiAC'
var = 'Q'
data = combined['sci_data'][chan][var]*changain[chan]
plotnow_azelsig(data, np.degrees(ra), np.degrees(dec), rt.nametochan(chan), var, res=1.0, minmax = [data.min()/5., data.max()/5.], radec=True)

NameError: name 'combined' is not defined

In [64]:
def plotnow_azelsig(data, az, el, chan, var, res, minmax=None, radec=False, supply_index=False):
    #flp=select_h5(fpath,yrmoday,st_hour,st_minute,ed_hour,ed_minute)
    #fld_demod, fld =select_dat(fpath,yrmoday,st_hour,st_minute,ed_hour,ed_minute)
    #i=0
    #while len(flp)<3:
    #	i+=1
    #	flp=select_h5(fpath,yrmoday,st_hour,int(st_minute)-i,ed_hour,int(ed_minute)+i)

    #pp=get_h5_pointing(flp)
    ##dd=get_demodulated_data_from_list(fld,supply_index=supply_index)
    #dd=get_all_demodulated_data(fld_demod, fld)	
    #combined=combine_cofe_h5_pointing(dd,pp)

    #synchronized data az and el values
    az1, el1 = az, el
    data = data

    #convert to temp for cryo sensors
    if chan == 12:
        data = convert.convert(data, 'i')
    if chan == 13:
        data = convert.convert(data, 'e')
    if chan == 14:
        data = convert.convert(data, 'h')
    if chan == 15:
        data = convert.convert(data, 'l')

    steps = len(data)

    #set az/el resolution
    dx = res
    dy = res

    #set up bins/grid
    x, y = np.arange(0., 360.+dx, dx), np.arange(0., 90. + dy, dy)
    AZ, EL = np.meshgrid(x, y)

    #small number for comparing floats
    epsilon = 1e-6

    #set up matrix for signal 
    z1 = np.zeros(len(x)*len(y))
    sig = np.reshape(z1, (len(y), len(x)))

    #set up matrix for keeping track of data points in single bin for averaging
    z2 = np.zeros(len(x)*len(y))
    count = np.reshape(z2, (len(y), len(x)))

    for i in range(steps):

        #round az/el points for comparison with grid	    
        el1[i] = rt.round_fraction(el1[i], dy)
        az1[i] = rt.round_fraction(az1[i], dx)  

        #find where data points belong in grid
        iel = np.where(abs(y - el1[i]) < epsilon)[0][0]
        iaz = np.where(abs(x - az1[i]) < epsilon)[0][0]

        #add 1 each time data point lands in same bin
        count[iel][iaz] += 1

        #add total number of data values in bin
        sig[iel][iaz] = sig[iel][iaz] + data[i]  

    #mask 0 count values so they dont show up in color plot
    count = ma.masked_where(count == 0.0, count)

    #take average of all data points in single bin
    sig = sig/count

    #change units on plot label
    if int(chan[2:]) < 12:
        unit = 'V'
    else:
        unit = 'K' 

    name = rt.chantoname(chan)

    plt.pcolormesh(AZ, EL, sig)
    plt.colorbar(label = 'Signal, %s' % unit)
    if minmax != None:
        plt.clim(minmax[0],minmax[1])
    else:  
        plt.clim(data1.min(),data1.max())
    if radec == False:
        plt.axis([AZ.min(), AZ.max(), EL.min(), EL.max()])
        plt.ylabel('elevation (deg)')
        plt.xlabel('azimuth (deg)')
    else:
        plt.axis([0., 360., -90, 90.])
        plt.xlabel('ra (deg)')
        plt.ylabel('dec (deg)')
    plt.title('%s %s data binned to azimuth and elevation' % (name, var))
    plt.grid()
    plt.show()

In [23]:
def plotnow_azrevsig(data, az, chan, var, res = 1.0,supply_index=False):
	#flp=select_h5(fpath,yrmoday,st_hour,st_minute,ed_hour,ed_minute)
	#fld_demod, fld =select_dat(fpath,yrmoday,st_hour,st_minute,ed_hour,ed_minute)
	#i=0
	#while len(flp)<3:
	#	i+=1
	#	flp=select_h5(fpath,yrmoday,st_hour,int(st_minute)-i,ed_hour,int(ed_minute)+i)

	#pp=get_h5_pointing(flp)
	#dd=get_demodulated_data_from_list(fld,supply_index=supply_index)
	#dd=get_all_demodulated_data(fld_demod, fld)
	#combined=combine_cofe_h5_pointing(dd,pp)
	
	#synchronized data and az values
	az1 = az
	data1 = data
	steps = len(data1)
	
	#convert to temp for cryo sensors
	if chan == 12:
		data1 = convert.convert(data1, 'i')
	if chan == 13:
		data1 = convert.convert(data1, 'e')
	if chan == 14:
		data1 = convert.convert(data1, 'h')
	if chan == 15:
		data1 = convert.convert(data1, 'l')
	
	#resolution
	dx = res
	dy = res
	
	#set up empty lists to append each revolution to
	data = []
	az = []
	iaz = [0]
	rev = 0
	
	#determine indices in azimuth/data array which correspond to a new revolution of the telescope
	for i in range(steps):
		#round values to resolution for comparison later
		az1[i] = round_fraction(az1[i], dx)
		if i > 0:
			if abs(az1[i] - az1[i-1]) >= 180.:
				iaz.append(i)
				rev += 1
	
	#append each revolution array to a list	    
	for j in range(rev):
		az.append(az1[iaz[j]:iaz[j+1]])
		data.append(data1[iaz[j]:iaz[j+1]])
	
	#append the last revolution
	data.append(data1[iaz[-1]:])
	az.append(az1[iaz[-1]:])
	rev += 1

	data = np.asarray(data)
	az = np.asarray(az)
	
	#create grid for plotting
	x, y = np.arange(0., 360.+dx, dx), np.arange(0., rev - 1 + dy, dy)
	AZ, REV = np.meshgrid(x, y)
	
	#set up empty array
	z = np.zeros(len(x)*len(y))
	sig = np.reshape(z, (len(y), len(x)))
	
	#small number for comparing floats
	epsilon = 1e-6
	
	#fill signal array with data points
	for r in range(rev):
		for a in range(len(x)):
			#find indices where combined azimuth data fits on x grid
			idx = np.where(abs(az[r] - x[a]) < epsilon)[0]
			#if idx length is 0 this will create a mask on that point, in idx len > 1, avg data points in the same bin
			sig[r][a] = data[r][idx].mean()
	
	#mask invalid values, i.e. where there are no data points
	sig = ma.masked_invalid(sig)
		
	#change units on plot label
	if int(chan[2:]) < 12:
		unit = 'V'
	else:
		unit = 'K' 

	name = rt.chantoname(chan)

	plt.pcolormesh(AZ, REV, sig)
	plt.colorbar(label = 'Signal, %s' % unit)
	plt.clim(data1.min(),data1.max())
	plt.axis([0., 360., 0., rev - 1])
	plt.ylabel('revolution #')
	plt.xlabel('azimuth (deg)')
	plt.title('%s %s data binned to azimuth and revolution #' % (name, var))
	plt.grid()
	plt.show()

In [61]:
def plotnow_azelsig2(data, az, el, chan, var, res, minmax=None, radec=False, supply_index=False):
    #flp=select_h5(fpath,yrmoday,st_hour,st_minute,ed_hour,ed_minute)
    #fld_demod, fld =select_dat(fpath,yrmoday,st_hour,st_minute,ed_hour,ed_minute)
    #i=0
    #while len(flp)<3:
    #	i+=1
    #	flp=select_h5(fpath,yrmoday,st_hour,int(st_minute)-i,ed_hour,int(ed_minute)+i)

    #pp=get_h5_pointing(flp)
    ##dd=get_demodulated_data_from_list(fld,supply_index=supply_index)
    #dd=get_all_demodulated_data(fld_demod, fld)	
    #combined=combine_cofe_h5_pointing(dd,pp)

    #synchronized data az and el values
    az1, el1 = az, el
    data = data

    #convert to temp for cryo sensors
    if chan == 12:
        data = convert.convert(data, 'i')
    if chan == 13:
        data = convert.convert(data, 'e')
    if chan == 14:
        data = convert.convert(data, 'h')
    if chan == 15:
        data = convert.convert(data, 'l')

    steps = len(data)

    #set az/el resolution
    dx = res
    dy = res

    #set up bins/grid
    x, y = np.arange(0., 360.+dx, dx), np.arange(0., 90. + dy, dy)
    AZ, EL = np.meshgrid(x, y)

    #small number for comparing floats
    epsilon = 1e-6

    #set up matrix for signal 
    z1 = np.zeros(len(x)*len(y))
    sig = np.reshape(z1, (len(y), len(x)))

    #set up matrix for keeping track of data points in single bin for averaging
    z2 = np.zeros(len(x)*len(y))
    count = np.reshape(z2, (len(y), len(x)))

    for i in range(steps):

        #round az/el points for comparison with grid	    
        el1[i] = rt.round_fraction(el1[i], dy)
        az1[i] = rt.round_fraction(az1[i], dx)  

        #find where data points belong in grid
        iel = np.where(abs(y - el1[i]) < epsilon)[0][0]
        iaz = np.where(abs(x - az1[i]) < epsilon)[0][0]

        #add 1 each time data point lands in same bin
        count[iel][iaz] += 1

        #add total number of data values in bin
        sig[iel][iaz] = sig[iel][iaz] + data[i]  

    #mask 0 count values so they dont show up in color plot
    count = ma.masked_where(count == 0.0, count)

    #take average of all data points in single bin
    sig = sig/count

    #change units on plot label
    if int(chan[2:]) < 12:
        unit = 'V'
    else:
        unit = 'K' 

    name = rt.chantoname(chan)

    plt.pcolormesh(AZ, EL, sig)
    plt.colorbar(label = 'Signal, %s' % unit)
    if minmax != None:
        plt.clim(minmax[0],minmax[1])
    else:  
        plt.clim(data1.min(),data1.max())
    if radec == False:
        plt.axis([AZ.min(), AZ.max(), EL.min(), EL.max()])
        plt.ylabel('elevation (deg)')
        plt.xlabel('azimuth (deg)')
    else:
        plt.axis([0., 360., -90, 90.])
        plt.xlabel('ra (deg)')
        plt.ylabel('dec (deg)')
    plt.title('%s %s data binned to azimuth and elevation' % (name, var))
    plt.grid()
    plt.show()